In [1]:
import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image
from torchvision.datasets import MNIST
import os

In [2]:
if not os.path.exists('/data/fjs/data/dc_img'):
    os.mkdir('/data/fjs/data/dc_img')

In [3]:
def to_img(x):
    x = 0.5 * (x + 1)
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 1, 28, 28)
    return x

In [5]:
num_epochs = 100
batch_size = 128
learning_rate = 1e-3

img_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

dataset = MNIST('/data/fjs/data', transform=img_transform,download=True)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

0.00B [00:00, ?B/s]

100%|█████████▉| 9.89M/9.91M [00:35<00:00, 359kB/s] 

Extracting /data/fjs/data/MNIST/raw/train-images-idx3-ubyte.gz



0.00B [00:00, ?B/s]


  0%|          | 0.00/28.9k [00:00<?, ?B/s]
 57%|█████▋    | 16.4k/28.9k [00:00<00:00, 65.6kB/s]
32.8kB [00:00, 44.3kB/s]                            
0.00B [00:00, ?B/s]

...


  0%|          | 0.00/1.65M [00:00<?, ?B/s]
  1%|          | 16.4k/1.65M [00:01<00:27, 59.7kB/s]
  2%|▏         | 41.0k/1.65M [00:01<00:23, 67.7kB/s]
  4%|▍         | 73.7k/1.65M [00:01<00:20, 78.2kB/s]
  8%|▊         | 131k/1.65M [00:01<00:15, 99.0kB/s] 
 10%|▉         | 164k/1.65M [00:02<00:13, 107kB/s] 
 12%|█▏        | 197k/1.65M [00:02<00:12, 115kB/s]
 14%|█▍        | 238k/1.65M [00:02<00:11, 127kB/s]
 17%|█▋        | 279k/1.65M [00:02<00:10, 137kB/s]
 20%|█▉        | 328k/1.65M [00:03<00:08, 151kB/s]
 22%|██▏       | 369k/1.65M [00:03<00:08, 156kB/s]
 25%|██▌       | 418k/1.65M [00:03<00:07, 166kB/s]
 28%|██▊       | 459k/1.65M [00:03<00:07, 168kB/s]
 31%|███       | 508k/1.65M [00:04<00:06, 176kB/s]
 34%|███▍      | 557k/1.65M [00:04<00:05, 182kB/s]
 35%|███▌      | 582k/1.65M [00:04<00:07, 145kB/s]
 39%|███▉      | 639k/1.65M [00:04<00:06, 166kB/s]
 41%|████      | 680k/1.65M [00:05<00:05, 164kB/s]
 43%|████▎     | 705k/1.65M [00:05<00:07, 132kB/s]
 45%|████▌     | 745k/1.65M 

Extracting /data/fjs/data/MNIST/raw/t10k-images-idx3-ubyte.gz




  0%|          | 0.00/4.54k [00:00<?, ?B/s]

8.19kB [00:00, 16.9kB/s]                   

Extracting /data/fjs/data/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


9.92MB [00:50, 359kB/s]                            
1.65MB [00:23, 210kB/s]                            

In [6]:
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 16, 3, stride=3, padding=1),  # b, 16, 10, 10
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=2),  # b, 16, 5, 5
            nn.Conv2d(16, 8, 3, stride=2, padding=1),  # b, 8, 3, 3
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=1)  # b, 8, 2, 2
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(8, 16, 3, stride=2),  # b, 16, 5, 5
            nn.ReLU(True),
            nn.ConvTranspose2d(16, 8, 5, stride=3, padding=1),  # b, 8, 15, 15
            nn.ReLU(True),
            nn.ConvTranspose2d(8, 1, 2, stride=2, padding=1),  # b, 1, 28, 28
            nn.Tanh()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [7]:
model = autoencoder().cuda()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay=1e-5)

In [16]:
for epoch in range(num_epochs):
    for data in dataloader:
        img, _ = data
        img = Variable(img).cuda()
        # ===================forward=====================
        output = model(img)
        loss = criterion(output, img)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'.format(epoch+1, num_epochs, loss.item()))
    #if epoch % 10 == 0:
        #pic = to_img(output.cpu().data)
        #save_image(pic, '/data/fjs/data/dc_img/image_{}.png'.format(epoch))

epoch [1/100], loss:0.1229
epoch [2/100], loss:0.1198
epoch [3/100], loss:0.1283
epoch [4/100], loss:0.1218
epoch [5/100], loss:0.1156
epoch [6/100], loss:0.1190
epoch [7/100], loss:0.1123
epoch [8/100], loss:0.1200
epoch [9/100], loss:0.1189
epoch [10/100], loss:0.1117
epoch [11/100], loss:0.1146
epoch [12/100], loss:0.1219
epoch [13/100], loss:0.1137
epoch [14/100], loss:0.1096
epoch [15/100], loss:0.1207
epoch [16/100], loss:0.1110
epoch [17/100], loss:0.1138
epoch [18/100], loss:0.1098
epoch [19/100], loss:0.1132
epoch [20/100], loss:0.1098
epoch [21/100], loss:0.1144
epoch [22/100], loss:0.1081
epoch [23/100], loss:0.1102
epoch [24/100], loss:0.1091
epoch [25/100], loss:0.1129
epoch [26/100], loss:0.1150
epoch [27/100], loss:0.1004
epoch [28/100], loss:0.1095
epoch [29/100], loss:0.1072
epoch [30/100], loss:0.1111
epoch [31/100], loss:0.0930
epoch [32/100], loss:0.1071
epoch [33/100], loss:0.1094
epoch [34/100], loss:0.0954
epoch [35/100], loss:0.1056
epoch [36/100], loss:0.1016
e